In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [3]:
# load data
train_df = pd.read_csv("./datasets/train.csv")
test_df = pd.read_csv("./datasets/test.csv")
combine = [train_df, test_df]

In [6]:
X_train = train_df.drop(["label"], axis=1)
y_train = train_df["label"]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

(37800, 784)